In [1004]:
import pandas as pd
from pandas.tseries.offsets import *
from sklearn import preprocessing
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import math
from copy import deepcopy
import json
from itertools import combinations, permutations

% matplotlib inline
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 500)

In [1005]:
def teacher_qc_score(start_time, end_time, df_qc, class_type_name_special, hq_name_special):
    # 筛选条件
    df_qc = df_qc.loc[(~df_qc['class_type_name'].isin(class_type_name_special)) & 
                    (~df_qc['hq_name'].isin(hq_name_special)) & 
                    (df_qc['start_time'] >= start_time) & (df_qc['end_time'] <= end_time)]
    # log score-mean
    df_qc['decay_index'] = ((end_time - df_qc['score_recorded_at']).dt.total_seconds()) / (3600 * 24) 
    df_qc.loc[df_qc['decay_index'] < 0, 'decay_index'] = 0 # qc时间有时会在end_time之后造成负值
    df_qc['decay_index'] = df_qc['decay_index'] + 2 # log计算，防止0值+1;取倒数+1，防止无穷情况
    df_qc['decay_index'] = df_qc['decay_index'].apply(lambda x: math.log(x, 10))
    df_qc['decay_index'] = 1 / df_qc['decay_index']
    df_qc['log_decay_score'] = df_qc['score'] * df_qc['decay_index']
    df_teacher_log_score = df_qc.groupby(['awj_teacher_id'], 
                        as_index=False).agg({'log_decay_score': np.sum, 'decay_index': np.sum})
    df_teacher_log_score['log_decay_score_mean'] = df_teacher_log_score['log_decay_score'] / df_teacher_log_score['decay_index']
    df_teacher_log_score = df_teacher_log_score[['awj_teacher_id', 'log_decay_score_mean']]
    # score
    df_teacher_score = df_qc.groupby(['awj_teacher_id'], 
                as_index=False)['score'].agg(['max', 'min', 'count', np.std]) 
    # std missing
    df_teacher_score.loc[df_teacher_score['std'].isnull(), 'std'] = df_teacher_score['std'].mean()
    df_teacher_score.rename(columns={'max': 'teacher_score_max', 'min': 'teacher_score_min', 
                        'count': 'teacher_qc_count', 'std': 'teacher_score_std'}, inplace=True)
    df_teacher_score.reset_index(inplace=True)
    # merge
    df_qc_res = pd.merge(df_teacher_score, df_teacher_log_score, on='awj_teacher_id', how='left')
    return df_qc_res

In [1006]:
def teacher_behavior(start_time, end_time, df_teacher_behavior, 
                         class_type_name_special, hq_name_special):
    # 筛选条件
    df_teacher_behavior = df_teacher_behavior.loc[
                    (~df_teacher_behavior['上课类型'].isin(class_type_name_special)) & 
                    (~df_teacher_behavior['机构'].isin(hq_name_special)) & 
                    (df_teacher_behavior['start_time'] >= start_time) & 
                    (df_teacher_behavior['end_time'] <= end_time) & 
                    (~df_teacher_behavior['teacher_status_for_lesson'].isin(['system_failure']))]
    # 预处理
    df_teacher_behavior['lesson_count'] = ((df_teacher_behavior['end_time'] - 
                        df_teacher_behavior['start_time']).dt.total_seconds()) / 3600 * 2
    df_teacher_behavior.loc[df_teacher_behavior['teacher_status_for_lesson'] == 'no_show', 
                            'ask_for_leave_advanced_minutes'] = 0
    df_teacher_behavior['ask_for_leave_advanced_days'] = df_teacher_behavior['ask_for_leave_advanced_minutes'] / (60 * 24)
    # 天数衰减
    df_teacher_behavior['decay_index'] = ((end_time - 
            df_teacher_behavior['start_time']).dt.total_seconds()) / (3600 * 24) 
    df_teacher_behavior.loc[df_teacher_behavior['decay_index'] < 0, 'decay_index'] = 0
    # log计算，防止0值+1;取倒数+1，防止无穷情况
    df_teacher_behavior['decay_index'] = df_teacher_behavior['decay_index'] + 2
    df_teacher_behavior['decay_index'] = df_teacher_behavior['decay_index'].apply(lambda x: math.log(x, 10))
    df_teacher_behavior['decay_index'] = 1 / df_teacher_behavior['decay_index']
    
    
    
    
    
    
    
    # count lesson types
    df_teacher_behavior_res = df_teacher_behavior[['awj_teacher_id']]
    df_teacher_behavior_res.drop_duplicates(keep='first', inplace=True)
    types = ['normal_lesson', 'late', 'no_show', 'abnormal_lesson', 'ask_for_leave']
    for itm in types:
        df_count = df_teacher_behavior.loc[
            df_teacher_behavior['teacher_status_for_lesson'].isin([itm])]
        df_count['log_lesson_count'] = df_count['lesson_count'] * df_count['decay_index']
        df_count = df_count.groupby(['awj_teacher_id'], as_index=False)['log_lesson_count'].sum()
        df_count.reset_index()
        if (itm.find('ask_for_leave') > -1) | (itm.find('lesson') > -1):
            df_count.rename(columns={'log_lesson_count': itm + '_log_count'}, inplace=True)
        else:
            df_count.rename(columns={'log_lesson_count': itm + '_lesson_log_count'}, inplace=True)
        df_teacher_behavior_res = pd.merge(df_teacher_behavior_res, df_count, 
                                          on='awj_teacher_id', how='left')
        
        
        
        
        
        
        
    # ask for leave advanced minutes
    df_advanced_days = df_teacher_behavior.loc[
        df_teacher_behavior['teacher_status_for_lesson'].isin(['ask_for_leave', 'no_show'])]  
    df_advanced_days['ask_for_leave_advanced_log_days'] = df_advanced_days['decay_index'] * df_advanced_days['ask_for_leave_advanced_days']
    # 求均值
    df_advanced_days_mean = df_advanced_days.groupby(['awj_teacher_id'], as_index=False).agg(
        {'ask_for_leave_advanced_log_days': np.sum, 'decay_index': np.sum})
    df_advanced_days_mean.reset_index(inplace=True)
    df_advanced_days_mean['advanced_days_log_mean'] = df_advanced_days_mean['ask_for_leave_advanced_log_days'] / df_advanced_days_mean['decay_index'] 
    df_advanced_days_mean = df_advanced_days_mean[['awj_teacher_id', 'advanced_days_log_mean']]
    # 求最大最小值及方差
    df_advanced_days_others = df_advanced_days.groupby(['awj_teacher_id'], 
            as_index=False)['ask_for_leave_advanced_days'].agg(['min', 'max', 'std'])
    df_advanced_days_others.reset_index(inplace=True)
    df_advanced_days_others.loc[df_advanced_days_others['std'].isnull(), 
                                'std'] = df_advanced_days_others['std'].mean()
    df_advanced_days_others.rename(columns={'min': 'advanced_days_min', 
                    'max': 'advanced_days_max', 'std': 'advanced_days_std'}, inplace=True)
    df_advanced_days = pd.merge(df_advanced_days_mean, df_advanced_days_others, 
                               on='awj_teacher_id', how='left')
    
    
    
    
    
    
    
    df_teacher_behavior_res = pd.merge(df_teacher_behavior_res, df_advanced_days, 
                                                       on='awj_teacher_id', how='left')
    # fill 0
    columns = ['normal_lesson_log_count', 'late_lesson_log_count', 'no_show_lesson_log_count', 
               'abnormal_lesson_log_count', 'ask_for_leave_log_count', 'advanced_days_log_mean', 
               'advanced_days_min', 'advanced_days_max', 'advanced_days_std']
    for itm in columns:
        df_teacher_behavior_res[itm].fillna(value=0, inplace=True)
    return df_teacher_behavior_res

In [1007]:
def stu_comment(start_time, end_time, df_stu_comment, class_type_name_special, hq_name_special):
    # 筛选条件：时间筛选条件由于筛选后数量过少，所以采用全量数据
    df_stu_comment = df_stu_comment.loc[~df_stu_comment['机构'].isin(hq_name_special)]
    # 天数衰减
    df_stu_comment['decay_index'] = ((end_time - 
            df_stu_comment['评价时间']).dt.total_seconds()) / (3600 * 24) 
    df_stu_comment.loc[df_stu_comment['decay_index'] < 0, 'decay_index'] = 0
    # log计算，防止0值+1;取倒数+1，防止无穷情况
    df_stu_comment['decay_index'] = df_stu_comment['decay_index'] + 2
    df_stu_comment['decay_index'] = df_stu_comment['decay_index'].apply(lambda x: math.log(x, 10))
    df_stu_comment['decay_index'] = 1 / df_stu_comment['decay_index']  
    # 正面标签统计:有时一次课有多个好评标签，导致有些老师上课次数很少，但好评标签总量超过次数本身，不公平
    # 所以此处每堂课不管有几个好评标签，都算做一个好评统计
    df_good_label = df_stu_comment.loc[(df_stu_comment['学生评价星级'].isin(['5-star', '4-star'])) & 
            (df_stu_comment['标签内容'].isin(['老师有耐心', '课堂氛围好', '课程生动有趣', '互动丰富']))]
    df_good_label = df_good_label.groupby(['评价id'], as_index=False).first()
    df_good_label['标签内容'] = 1
    df_good_label['标签内容_processed'] = df_good_label['标签内容'] * df_good_label['decay_index']
    df_good_label = df_good_label.groupby(['awj_teacher_id'], 
                            as_index=False)['标签内容_processed'].sum()
    df_good_label.rename(columns={'标签内容_processed': 'stu_comment_log_good_behavior'}, inplace=True)
    # 负面标签统计：主观原因。客观原因---产品说环境原因可能是因为是视频供应商平台或学生自己网络问题，所以不应该计入
    # 去除4星及5星的数据，理论上，四星五星无负面评价，有的都是系统bug导致
    df_bad_label = df_stu_comment.loc[(~df_stu_comment['学生评价星级'].isin(['5-star', '4-star'])) & 
                    (df_stu_comment['标签内容'].isin(['老师语速过快', '互动较少', 
                    '课堂氛围差', '未及时纠正错误', '老师缺乏耐心']))]
    df_bad_label = df_bad_label.groupby(['评价id'], as_index=False).first()
    df_bad_label['标签内容'] = 1
    df_bad_label['标签内容_processed'] = df_bad_label['标签内容'] * df_bad_label['decay_index']
    df_bad_label = df_bad_label.groupby(['awj_teacher_id'], 
                                as_index=False)['标签内容_processed'].sum()
    df_bad_label.rename(columns={'标签内容_processed': 'stu_comment_log_bad_behavior'}, inplace=True)
    # merge
    df_stu_comment_res = pd.merge(df_good_label, df_bad_label, on='awj_teacher_id', how='outer')
    df_stu_comment_res['stu_comment_log_bad_behavior'].fillna(value=0, inplace=True)
    df_stu_comment_res['stu_comment_log_good_behavior'].fillna(value=0, inplace=True)
    return df_stu_comment_res

In [1008]:
if __name__ == '__main__':
    path = path = '/Users/roger.zhou/Downloads/星级老师/'
    end_time = pd.to_datetime(datetime.datetime.now().date()) + MonthEnd(n=-1) + DateOffset(hours=23.99999)
    start_time = end_time - DateOffset(months=3)
    class_type_name_special = ['Demo', '补课(非爱乐奇直属老师)', '托福班（30刀）', 'TOFEL', 
                              'VIP Writing/TOFEL（35刀）', 'Elite Pilot', 'Feeback Session', 
                              'New Teacher Test Class', '补课(爱乐奇直属老师)', 'Test Class', 
                              'Academic Meeting (Long)', 'Cur Experience Session-S', 
                              'Training-receiving', 'Cur Experience Session-L', 
                              'VIP Writing/TOFEL', 'Orientation Class', 'Academic Meeting', 
                              'Experience-receiving']
    hq_name_special = ['test']
    print('start_time:', start_time, '\n', 'end_time:', end_time, '\n')
    
    ##### 老师监控表 #####
    df_teacher_monitoring = pd.read_csv(path + 'awj_teacher_monitoring.csv', encoding='utf-8', sep=',')
    # dtypes
    df_teacher_monitoring['awj_teacher_id'] = df_teacher_monitoring['awj_teacher_id'].astype('int')
    df_teacher_monitoring['created_at'] = pd.to_datetime(df_teacher_monitoring['created_at'])
    # drop duplicates
    df_teacher_monitoring.drop_duplicates(
                        subset=list(df_teacher_monitoring.columns), keep='first', inplace=True)
    # sort
    df_teacher_monitoring = df_teacher_monitoring.sort_values(
                                by=['awj_teacher_id', 'created_at'], ascending=[1, 1])
    # 该表只取abnormal_type为4，5的行，分别表示zoom崩溃和课件崩溃，非老师原因
    df_teacher_monitoring = df_teacher_monitoring.loc[
                    df_teacher_monitoring['abnormal_type'].isin([4, 5])]
    df_teacher_monitoring = df_teacher_monitoring.groupby(
                ['awj_teacher_id', 'awjcls_lesson_id', 'abnormal_type'], as_index=False).last()
    # 唯一主键awj_teacher_id+awjcls_lesson_id check
    cache = df_teacher_monitoring.groupby(['awj_teacher_id', 'awjcls_lesson_id'], 
                                      as_index=False)['abnormal_type'].count()
    print('df_teacher_monitoring key check_duplicate no(awj_teacher_id, awjcls_lesson_id):', 
          cache.loc[cache['abnormal_type'] > 1].shape[0], '\n')
    
    ##### 老师QC明细表 #####
    df_qc = pd.read_csv(path + '老师qc明细.csv', encoding='utf-8', sep=',')
    # dtypes
    df_qc['awj_teacher_id'] = df_qc['awj_teacher_id'].astype('int')
    df_qc['score_recorded_at'] = pd.to_datetime(df_qc['score_recorded_at'])
    df_qc['assigned_at'] = pd.to_datetime(df_qc['assigned_at'])
    df_qc['start_time'] = pd.to_datetime(df_qc['start_time'])
    df_qc['end_time'] = pd.to_datetime(df_qc['end_time'])
    print('df_qc_before drop duplicates:', df_qc.shape)
    # drop duplicates-老师同一堂课有时会有多次qc，check后发现分数都一样，所以去重时按照一下字段去重即可
    columns = ['awj_teacher_id', 'awjcls_lesson_id', 'score']
    df_qc.drop_duplicates(subset=columns, keep='first', inplace=True)
    print('df_qc_drop duplicates:', df_qc.shape)
    # sort
    df_qc = df_qc.sort_values(by=['awj_teacher_id', 'start_time'], ascending=[1, 1])
    # 唯一主键awj_teacher_id+awjcls_lesson_id check
    cache = df_qc.groupby(['awj_teacher_id', 'awjcls_lesson_id'], 
                                      as_index=False)['teacher_name'].count()
    print('df_qc key check_duplicate no(awj_teacher_id, awjcls_lesson_id):', 
          cache.loc[cache['teacher_name'] > 1].shape[0], '\n')
    
    ##### 老师行为表 #####
    df_teacher_behavior = pd.read_csv(path + '老师行为信息明细.csv', sep=',', encoding='utf-8')
    # dtypes
    df_teacher_behavior['awj_teacher_id'] = df_teacher_behavior['awj_teacher_id'].astype('int')
    df_teacher_behavior['start_time'] = pd.to_datetime(df_teacher_behavior['start_time'])
    df_teacher_behavior['end_time'] = pd.to_datetime(df_teacher_behavior['end_time'])
    df_teacher_behavior['actual_start_time'] = pd.to_datetime(df_teacher_behavior['actual_start_time'])
    df_teacher_behavior['actual_end_time'] = pd.to_datetime(df_teacher_behavior['actual_end_time'])
    df_teacher_behavior.drop_duplicates(subset=list(df_teacher_behavior.columns), inplace=True)
    # sort
    df_teacher_behavior = df_teacher_behavior.sort_values(by=[
            'awj_teacher_id', 'awj_lesson_id', 'start_time', '积分变化'], ascending=[1, 1, 1, 1])
    print('df_teacher_behavior shape:', df_teacher_behavior.shape)
    # 老师id加lesson_id应该唯一，但有时有重复情况，因为有些老师先请了假，后来又来上课了，所以请假应该去除
    df_teacher_behavior = df_teacher_behavior.groupby(
                ['awj_teacher_id', 'awj_lesson_id'], as_index=False).last()
    print('df_teacher_behavior shape(teacher_id+lesson_id as key:)', df_teacher_behavior.shape)
    # 与monitoring表对比，去除老师abnormal_lesson细分为4，5状态下的课程记录
    df_teacher_behavior = pd.merge(df_teacher_behavior, 
                    df_teacher_monitoring[['awj_teacher_id', 'awjcls_lesson_id', 'abnormal_type']], 
                    left_on=['awj_teacher_id', 'awj_lesson_id'], 
                    right_on=['awj_teacher_id', 'awjcls_lesson_id'], how='left')
    index = df_teacher_behavior.loc[
                    (df_teacher_behavior['teacher_status_for_lesson'] == 'abnormal_lesson') & 
                    (df_teacher_behavior['abnormal_type'].isin([4, 5]))].index
    df_teacher_behavior = df_teacher_behavior.loc[~df_teacher_behavior.index.isin(list(index))]
    df_teacher_behavior.drop(['awjcls_lesson_id', 'abnormal_type'], axis=1, inplace=True)
    print('df_teacher_behavior shape(no abnormal_type=4, 5)', df_teacher_behavior.shape)
    # 唯一主键awj_teacher_id+awjcls_lesson_id check
    cache = df_teacher_behavior.groupby(['awj_teacher_id', 'awj_lesson_id'], 
                                      as_index=False)['start_time'].count()
    print('df_teacher_behavior key check_duplicate no(awj_teacher_id, awj_lesson_id):', 
          cache.loc[cache['start_time'] > 1].shape[0], '\n')
    
    ##### 老师信息表 #####
    df_teacher_info = pd.read_csv(path + '老师基本信息.csv', sep=',', encoding='utf-8')
    # dtypes
    df_teacher_info['awj_teacher_id'] = df_teacher_info['awj_teacher_id'].astype(int)
    df_teacher_info['创建时间'] = pd.to_datetime(df_teacher_info['创建时间'])
    df_teacher_info['首次上架时间'] = pd.to_datetime(df_teacher_info['首次上架时间'])
    df_teacher_info['首课时间'] = pd.to_datetime(df_teacher_info['首课时间'])
    df_teacher_info.drop_duplicates(subset=list(df_teacher_info.columns), 
                                                    keep='first', inplace=True)
    # 根据业务要求去除只取某些type类型老师，其他去除
    df_teacher_info = df_teacher_info.loc[df_teacher_info['teacher_type'].isin([
                'booking&arrangement', 'arrangement_only', 'booking_only'])]
    # sort
    df_teacher_info = df_teacher_info.sort_values(by=['awj_teacher_id'], ascending=[1])
    df_teacher_info = df_teacher_info[['awj_teacher_id', 'state', '创建时间', 
                                                           '首次上架时间', '首课时间']]
    print('df_teacher_info key check duplicate no(awj_teacher_id):', 
          df_teacher_info.shape[0] - df_teacher_info['awj_teacher_id'].nunique(), '\n')
    
    ##### 学生评价明细表(去重在函数内完成) #####
    df_stu_comment = pd.read_csv(path + '学生评价明细.csv', sep=',', encoding='utf-8')
    # dtypes
    df_stu_comment['awj_teacher_id'] = df_stu_comment['awj_teacher_id'].astype(int)
    df_stu_comment['评价时间'] = pd.to_datetime(df_stu_comment['评价时间'])
    df_stu_comment.drop_duplicates(subset=list(df_stu_comment.columns), keep='first', inplace=True)
    # sort
    df_stu_comment = df_stu_comment.sort_values(by=[
                    'awj_teacher_id', '评价时间'], ascending=[1, 1])  
    
    # 函数调用
    df_qc_res = teacher_qc_score(start_time, end_time, df_qc, 
                                 class_type_name_special, hq_name_special)
    df_teacher_behavior_res = teacher_behavior(start_time, 
                        end_time, df_teacher_behavior, class_type_name_special, hq_name_special)
    df_stu_comment_res = stu_comment(start_time, end_time, df_stu_comment, 
                                     class_type_name_special, hq_name_special)
    # 宽表
    df_wide = pd.merge(df_teacher_info, df_teacher_behavior_res, on='awj_teacher_id', how='left')
    df_wide = pd.merge(df_wide, df_qc_res, on='awj_teacher_id', how='left')
    df_wide = pd.merge(df_wide, df_stu_comment_res, on='awj_teacher_id', how='left')
    # 学生评价---采用全量数据，需用到全量老师上课数值
    df_lesson_count_all = teacher_behavior(end_time - DateOffset(months=360), end_time, 
                            df_teacher_behavior, class_type_name_special, hq_name_special)
    df_lesson_count_all = df_lesson_count_all[['awj_teacher_id', 'normal_lesson_log_count', 
                                        'late_lesson_log_count', 'no_show_lesson_log_count', 
                                        'ask_for_leave_log_count', 'abnormal_lesson_log_count']]
    df_lesson_count_all.rename(columns={'normal_lesson_log_count': 'normal_lesson_log_count_all', 
                                    'late_lesson_log_count': 'late_lesson_log_count_all', 
                                    'no_show_lesson_log_count': 'no_show_lesson_log_count_all', 
                                    'ask_for_leave_log_count': 'ask_for_leave_log_count_all', 
                                    'abnormal_lesson_log_count': 'abnormal_lesson_log_count_all'}, 
                                    inplace=True)
    df_wide = pd.merge(df_wide, df_lesson_count_all, on='awj_teacher_id', how='left')
    # 衍生新字段
    df_wide['log_ask_for_leave/log_normal_lesson'] = df_wide['ask_for_leave_log_count'] / df_wide['normal_lesson_log_count']
    df_wide['abnormal_all_log_count'] = (df_wide['no_show_lesson_log_count'] + 
                                      df_wide['late_lesson_log_count'] + 
                                      df_wide['abnormal_lesson_log_count'] + 
                                      df_wide['ask_for_leave_log_count'])
    df_wide['abnormal_all_log_percent'] = (df_wide['abnormal_all_log_count']) / (
                                      df_wide['normal_lesson_log_count'] + 
                                      df_wide['abnormal_all_log_count'])
    df_wide['lesson_time_range'] = ((end_time - df_wide['首课时间']).dt.total_seconds()) / (3600 * 24) 
    df_wide.loc[df_wide['lesson_time_range'] == 0, 'lesson_time_range'] = ((end_time - df_wide['创建时间']).dt.total_seconds()) / (3600 * 24)  
    # 判断是否是新老师
    df_wide['old_new_teacher'] = 'old'
    df_wide.loc[(df_wide['首课时间'].isnull()) & 
                (df_wide['state'].isin(['oboard', 'active'])), 'old_new_teacher'] = 'new'
    # 缺失值填补
    # 无用字段去除
    df_wide.drop(['创建时间', '首次上架时间', '首课时间', 'state'], axis=1, inplace=True)
    columns = list(df_wide.columns)
    columns.pop(columns.index('awj_teacher_id'))
    columns.pop(columns.index('old_new_teacher'))
    # new teacher: mean
    for itm in columns:
        df_wide.loc[df_wide['old_new_teacher'] == 'new', itm] = df_wide.loc[
            (df_wide['normal_lesson_log_count'] > 0), itm].mean()
    # old teacher:0
    df_wide.fillna(value=0, inplace=True)
    new_teacher = df_wide.loc[df_wide['old_new_teacher'] == 'new', 'awj_teacher_id']
    # 有些老师没有请过假，advanced_days字段为0，填为均值
    df_wide.loc[(df_wide['ask_for_leave_log_count'] == 0) & (df_wide['normal_lesson_log_count'] > 0), 
        'advanced_days_max'] = df_wide.loc[(df_wide['advanced_days_max'] != 0) & 
                        df_wide['normal_lesson_log_count'] > 0, 'advanced_days_max'].mean()
    df_wide.loc[(df_wide['ask_for_leave_log_count'] == 0) & (df_wide['normal_lesson_log_count'] > 0), 
        'advanced_days_log_mean'] = df_wide.loc[(df_wide['advanced_days_log_mean'] != 0) & 
                        df_wide['normal_lesson_log_count'] > 0, 'advanced_days_log_mean'].mean()
    # 平滑
    smooth = ['late_lesson_log_count', 'no_show_lesson_log_count', 'abnormal_lesson_log_count', 
              'ask_for_leave_log_count', 'stu_comment_log_bad_behavior']
    for itm in smooth:
        df_wide[itm] = df_wide[itm] + math.log(2, 10)
    # 字段处理
    # 比例计算
    df_wide['late_lesson_log_percent'] = df_wide['late_lesson_log_count'] / df_wide['normal_lesson_log_count']
    df_wide['no_show_lesson_log_percent'] = df_wide['no_show_lesson_log_count'] / df_wide['normal_lesson_log_count']
    df_wide['abnormal_lesson_log_percent'] = df_wide['abnormal_lesson_log_count'] / df_wide['normal_lesson_log_count']
    df_wide['ask_for_leave_log_percent'] = df_wide['ask_for_leave_log_count'] / df_wide['normal_lesson_log_count']
    # 查看上课异常行为的分位数
    columns = ['no_show_lesson_log_percent', 'late_lesson_log_percent', 
               'abnormal_lesson_log_percent', 'ask_for_leave_log_percent', 
               'abnormal_all_log_percent', 'log_ask_for_leave/log_normal_lesson', 
               'advanced_days_log_mean', 'advanced_days_max']
    quantiles = [0.1, 0.15, 0.2, 0.25, 0.3, 0.33, 0.5, 0.55, 0.6, 0.66, 0.7, 0.75, 
                 0.77, 0.8, 0.83, 0.85, 0.9, 0.92, 0.95, 1]
    for i in range(len(columns)):
        print('\n', columns[i], ':')
        for itm in quantiles:
            print(str(itm), ':', df_wide.loc[df_wide['normal_lesson_log_count'] > 0, 
                                                        columns[i]].quantile(itm))
    # 老师异常行为有一项出现较大异常值时或整体较差，normal_lesson_log_count降为相应较低数值，整体表现变差
    counts = ['ask_for_leave_log_count', 'late_lesson_log_count', 
               'no_show_lesson_log_count', 'abnormal_lesson_log_count', 
               'abnormal_all_log_count']
    columns = ['ask_for_leave_log_percent', 'late_lesson_log_percent', 
               'no_show_lesson_log_percent', 'abnormal_lesson_log_percent', 
               'abnormal_all_log_percent']
    cache_columns = ['normal_lesson_log_count_processed_ask_for_leave_cahce1', 
                     'normal_lesson_log_count_processed_late_lesson_cache2', 
                     'normal_lesson_log_count_processed_no_show_cache3', 
                     'normal_lesson_log_count_processed_abnormal_lesson_cache4', 
                     'normal_lesson_log_count_processed_abnormal_all_cahce5'
                    ]
    for itm in cache_columns:
        df_wide[itm] = df_wide['normal_lesson_log_count']
    df_wide['normal_lesson_log_count_processed'] = df_wide['normal_lesson_log_count']
    # 分位数
    quantiles = [[0.6, 0.7, 0.75, 0.8, 1], [0.8, 0.9, 0.92, 0.95, 1], 
                [0.5, 0.6, 0.66, 0.75, 0.8, 0.85, 0.9, 1], [0.8, 0.85, 0.9, 0.95, 1], 
                [0.55, 0.66, 0.75, 0.85, 1]]
    # 降低比例
    indexes = [[0.8, 0.6, 0.3, 0.1], [0.8, 0.7, 0.3, 0.1], 
              [0.8, 0.75, 0.6, 0.5, 0.3, 0.1, 0.03], [0.9, 0.85, 0.7, 0.3], 
              [0.8, 0.7, 0.3, 0.1]]
    for i in range(len(columns)):
        for k in range(len(quantiles[i]) - 1):
            standard1 = df_wide.loc[df_wide['normal_lesson_log_count'] > 0, 
                                    columns[i]].quantile(quantiles[i][k])
            standard2 = df_wide.loc[df_wide['normal_lesson_log_count'] > 0, 
                                    columns[i]].quantile(quantiles[i][k + 1])
            # 降低正常上课的数量
            df_wide.loc[(df_wide['normal_lesson_log_count'] > 0) & 
                (df_wide[columns[i]] > standard1) & (df_wide[columns[i]] <= standard2), 
                cache_columns[i]] = df_wide['normal_lesson_log_count'] * indexes[i][k]
            # 新老师因为平滑原因，所以再复原，不惩罚
            df_wide.loc[(df_wide['normal_lesson_log_count'] > 0) & 
                (df_wide[columns[i]] > standard1) & (df_wide[columns[i]] <= standard2) & 
                (df_wide[counts[i]] == math.log(2, 10)) & 
                (df_wide['normal_lesson_log_count'] <= 8), 
                cache_columns[i]] = df_wide['normal_lesson_log_count']
    # 从5行cache_colume中取最小值
    df_wide['normal_lesson_log_count_processed'] = df_wide[cache_columns].min(axis=1)
    # 大小方向统一化
    # 重新计算四个percent
    df_wide['late_lesson_log_percent_processed'] = df_wide['late_lesson_log_count'] / df_wide['normal_lesson_log_count_processed']
    df_wide['no_show_lesson_log_percent_processed'] = df_wide['no_show_lesson_log_count'] / df_wide['normal_lesson_log_count_processed']
    df_wide['abnormal_lesson_log_percent_processed'] = df_wide['abnormal_lesson_log_count'] / df_wide['normal_lesson_log_count_processed']
    df_wide['ask_for_leave_log_percent_processed'] = df_wide['ask_for_leave_log_count'] / df_wide['normal_lesson_log_count_processed']
    df_wide['normal_log_lesson_per_week'] = df_wide['normal_lesson_log_count_processed'] / (df_wide['lesson_time_range']) 
    # 取倒数
    rcp = ['late_lesson_log_processed', 'no_show_lesson_log_processed', 
           'abnormal_lesson_log_processed', 'ask_for_leave_log_processed', 
           'stu_comment_log_bad_behavior_processed']
    cols = ['late_lesson_log_percent_processed', 'no_show_lesson_log_percent_processed', 
            'abnormal_lesson_log_percent_processed', 'ask_for_leave_log_percent_processed', 
            'stu_comment_log_bad_behavior']
    for i in range(len(rcp)):
        df_wide[rcp[i]] = 1 / df_wide[cols[i]]
    # 修正老师上课数量少但好评较多的情况（如老师id642）
    df_wide['stu_comment_log_good_behavior_processed'] = df_wide['stu_comment_log_good_behavior'] / ( 
                                df_wide['normal_lesson_log_count_all']
                                + df_wide['late_lesson_log_count_all'] 
                                + df_wide['no_show_lesson_log_count_all'] 
                                + df_wide['ask_for_leave_log_count_all'] 
                                + df_wide['abnormal_lesson_log_count_all'])                    
    df_wide['stu_comment_log_bad_behavior_processed'] = df_wide['stu_comment_log_bad_behavior'] / (
                                df_wide['normal_lesson_log_count_all']
#                                 + df_wide['late_lesson_log_count_all'] 
#                                 + df_wide['no_show_lesson_log_count_all'] 
#                                 + df_wide['ask_for_leave_log_count_all'] 
#                                 + df_wide['abnormal_lesson_log_count_all']
                                    )
    # 修正老师请假次数过多，但提前请假天数指标过好的情况（如老师id642） 
    columns = ['advanced_days_log_mean', 'advanced_days_max']
    quantiles = [0.6, 0.66, 0.7, 0.75, 0.77, 0.8, 0.85, 1]
    indexes = [0.9, 0.8, 0.7, 0.5, 0.3, 0.1, 0.05]
    for i in range(len(columns)):
        for k in range(len(quantiles) - 1):
            percent1 = df_wide.loc[df_wide['normal_lesson_log_count'] > 0, 
                                'log_ask_for_leave/log_normal_lesson'].quantile(quantiles[k])
            percent2 = df_wide.loc[df_wide['normal_lesson_log_count'] > 0, 
                                'log_ask_for_leave/log_normal_lesson'].quantile(quantiles[k + 1])
            df_wide.loc[(df_wide['normal_lesson_log_count'] > 0) & 
                        (df_wide['log_ask_for_leave/log_normal_lesson'] > percent1) & 
                        (df_wide['log_ask_for_leave/log_normal_lesson'] <= percent2), 
                        columns[i]] = df_wide[columns[i]] * indexes[k]
    # 无上课记录老师
    columns = list(df_wide.columns)
    columns.pop(columns.index('awj_teacher_id'))
    for itm in columns:
        df_wide.loc[df_wide['normal_lesson_log_count'] == 0, itm] = 0
    # 只取有行为数据的
    df_wide_final = df_wide.loc[df_wide['normal_lesson_log_count'] > 0]
    # delete columns
    df_wide_final.drop(['advanced_days_std', 'teacher_score_std', 
                  'advanced_days_min', 'lesson_time_range', 
                  'normal_lesson_log_count_all', 'no_show_lesson_log_count_all', 
                  'abnormal_lesson_log_count_all', 'late_lesson_log_count_all', 
                  'ask_for_leave_log_count_all', 'old_new_teacher', 
                  'teacher_qc_count', 'normal_lesson_log_count', 'late_lesson_log_count', 
                  'no_show_lesson_log_count', 'abnormal_lesson_log_count', 
                  'ask_for_leave_log_count', 'stu_comment_log_good_behavior', 
                  'stu_comment_log_bad_behavior', 
                  'log_ask_for_leave/log_normal_lesson',  'abnormal_lesson_log_percent',
                  'late_lesson_log_percent', 'no_show_lesson_log_percent', 
                  'ask_for_leave_log_percent', 'abnormal_all_log_count', 
                  'late_lesson_log_percent_processed', 'no_show_lesson_log_percent_processed', 
                  'abnormal_lesson_log_percent_processed', 'ask_for_leave_log_percent_processed', 
                  'normal_lesson_log_count_processed', 'abnormal_all_log_percent', 
                  'normal_lesson_log_count_processed_ask_for_leave_cahce1', 
                  'normal_lesson_log_count_processed_late_lesson_cache2', 
                  'normal_lesson_log_count_processed_no_show_cache3', 
                  'normal_lesson_log_count_processed_abnormal_lesson_cache4', 
                  'normal_lesson_log_count_processed_abnormal_all_cahce5',                                        
                  'normal_log_lesson_per_week', 
                  'stu_comment_log_bad_behavior_processed'], 
                   axis=1, inplace=True)
    # save
    df_wide_final.to_csv(path + 'df_wide_log_final.csv', sep=',', index=False, encoding='utf-8')

start_time: 2018-01-30 23:59:59.964000 
 end_time: 2018-04-30 23:59:59.964000 

df_teacher_monitoring key check_duplicate no(awj_teacher_id, awjcls_lesson_id): 0 

df_qc_before drop duplicates: (29311, 17)
df_qc_drop duplicates: (24184, 17)
df_qc key check_duplicate no(awj_teacher_id, awjcls_lesson_id): 0 

df_teacher_behavior shape: (241621, 13)
df_teacher_behavior shape(teacher_id+lesson_id as key:) (241469, 13)
df_teacher_behavior shape(no abnormal_type=4, 5) (241467, 13)
df_teacher_behavior key check_duplicate no(awj_teacher_id, awj_lesson_id): 0 

df_teacher_info key check duplicate no(awj_teacher_id): 0 



/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_


 no_show_lesson_log_percent :
0.1 : 0.0017861690310692672
0.15 : 0.002330144202808163
0.2 : 0.0029021563036939345
0.25 : 0.003482588679703572
0.3 : 0.004098390217401592
0.33 : 0.004665703610870716
0.5 : 0.01054202982440932
0.55 : 0.012818233492788964
0.6 : 0.012832725816617648
0.66 : 0.014951704763786688
0.7 : 0.01846137967319565
0.75 : 0.02972795413302779
0.77 : 0.03313220669342654
0.8 : 0.03820796181046023
0.83 : 0.045262864422940044
0.85 : 0.05877166648082807
0.9 : 0.09928910002012299
0.92 : 0.12949367713011428
0.95 : 0.2660843838119369
1 : 25.901074963742026

 late_lesson_log_percent :
0.1 : 0.0018666196682732292
0.15 : 0.002294058535537647
0.2 : 0.002680781955185766
0.25 : 0.0031247849804458077
0.3 : 0.0036492894179306415
0.33 : 0.004114415749656181
0.5 : 0.010127500480385179
0.55 : 0.012926512259553491
0.6 : 0.016083105469158032
0.66 : 0.0243806899156339
0.7 : 0.027049405497154167
0.75 : 0.027432159700350782
0.77 : 0.033138320012811084
0.8 : 0.038847984811040494
0.83 : 0.0452857

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:319: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [1012]:
# FA_indexs
fa_index = pd.read_excel(path + 'fa_indexs.xlsx')
fa_index

,fa_index1,fa_index2,fa_index3,fa_index4
0,-0.067965,0.007287,0.535777,0.057239
1,-0.081179,0.007034,0.541467,0.043481
2,-0.021609,0.338660,0.003847,-0.008771
3,-0.046738,0.344282,0.012750,-0.010007
4,-0.035079,0.348230,0.003747,-0.018199
5,0.283483,-0.024297,-0.030890,-0.024959
6,0.318853,-0.032983,-0.056064,0.024066
7,0.307936,-0.026679,-0.050784,0.083130
8,0.293590,-0.038004,-0.083815,-0.077168
9,0.002655,-0.017316,0.062766,1.001195


In [1013]:
fa_index = fa_index.as_matrix()
df_wide_matrix = deepcopy(df_wide_final)
df_wide_matrix.drop(['awj_teacher_id'], axis=1, inplace=True)
df_wide_matrix = df_wide_matrix.as_matrix()
# df_wide标准化
df_wide_matrix = preprocessing.scale(df_wide_matrix)
# 每个老师的各因子得分
fa_score = np.dot(df_wide_matrix, fa_index)
# 主成分贡献率
var = np.array([[0.37591 / 0.88651], 
                [0.26337 / 0.88651], 
                [0.15250 / 0.88651], 
                [0.09473 / 0.88651]])
# 每个老师的最终得分
final_score = np.dot(fa_score, var)

In [1015]:
# df格式
teacher_fa_score = np.hstack((fa_score, final_score))
teacher_fa_score = pd.DataFrame(teacher_fa_score)
teacher_fa_score['awj_teacher_id'] = list(df_wide_final['awj_teacher_id'])
teacher_fa_score.rename(columns={0: 'teacher_behavior_score', 1: 'teacher_qc_score', 
                                 2:'teacher_attitude_score', 3: 'student_comment_good_score', 
                                 4: 'final_score'}, inplace=True)
# 星级映射(去除过去一段时间周期内没上过课的老师)
teacher_fa_score = teacher_fa_score.sort_values(by='final_score', ascending=0)
# 业务要求的分位数
star_5 = teacher_fa_score['final_score'].quantile(0.8)
star_4 = teacher_fa_score['final_score'].quantile(0.5)
star_3 = teacher_fa_score['final_score'].quantile(0.2)
star_2 = teacher_fa_score['final_score'].quantile(0.1)
print('5:', star_5, '4:', star_4, '3:', star_3, '2:', star_2)
teacher_fa_score.loc[teacher_fa_score['final_score'] <= star_2, 'star'] = 1
teacher_fa_score.loc[(teacher_fa_score['final_score'] > star_2) & 
                     (teacher_fa_score['final_score'] <= star_3), 'star'] = 2
teacher_fa_score.loc[(teacher_fa_score['final_score'] > star_3) & 
                     (teacher_fa_score['final_score'] <= star_4), 'star'] = 3
teacher_fa_score.loc[(teacher_fa_score['final_score'] > star_4) & 
                     (teacher_fa_score['final_score'] <= star_5), 'star'] = 4
teacher_fa_score.loc[teacher_fa_score['final_score'] > star_5, 'star'] = 5
# 拼回去
teacher_fa_score = pd.merge(teacher_fa_score, df_wide[
    ['awj_teacher_id', 'normal_lesson_log_count']], on='awj_teacher_id', how='right')
teacher_fa_score.fillna(value=0, inplace=True)
# 没上过课的老老师都统一补为3星
teacher_fa_score['star'] = teacher_fa_score['star'].replace({0: 3})
teacher_fa_score.to_csv(path + 'teacher_star.csv', sep=',', encoding='utf-8', index=False)

5: 0.39932723159523076 4: -0.06554925390984362 3: -0.3866192561701031 2: -0.47629323522986805
